### Get ready

In [ ]:
import os
import glob
from pathlib import Path

import numpy as np
from scipy.stats import rankdata, ttest_rel, ttest_1samp

from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

import pandas as pd
import seaborn as sns

import nibabel as nib
from nilearn.maskers import NiftiLabelsMasker
from nilearn.plotting import plot_glass_brain, plot_stat_map, view_img, view_img_on_surf

from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import fdr, threshold

from sklearn.metrics import pairwise_distances
from sklearn.utils import check_random_state
from sklearn.manifold import TSNE

import datalad.api as dl

from copy import deepcopy
from scipy.spatial.distance import pdist
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import stats
from scipy.spatial import distance
from scipy.spatial.distance import squareform

from nltools.utils import get_resource_path
from nltools.file_reader import onsets_to_dm
from nltools.data import Design_Matrix
from nltools.mask import create_sphere
from nltools.mask import expand_mask, collapse_mask

In [ ]:
## bring in behavior data

data_dir = '#directory path'
behav_data = pd.read_csv(os.path.join(data_dir, '#behav file name.csv'), sep = ',') # file should contain sub & behav scores
behav_data.head()

In [ ]:
subj_data = behav_data["sub"]
subj_list = subj_data

subj_list

In [ ]:
## bring in FC similarity data 
    # should be symmetrical, n x n (n is number of sub) format
    # contain every pairwise similarity comparison with others (diagonal should be 1)
data = pd.read_csv(os.path.join(data_dir,'#similarity file name.csv'), header=None, index_col=None, sep = ',')

In [ ]:
## visualization
data_np = np.array(data)
fconn = Adjacency(data_np, matrix_type = 'similarity')

fconn.plot(cmap='RdYlBu_r') 
plt.savefig('fconn.jpg', format='jpeg', dpi=300)

### IS-RSA ###

run with AnnaK framework

In [ ]:
behav = behav_data["GRIT_total"]
behav_rank = rankdata(behav) # convert the raw scores to ranks

def sort_square_mtx(mtx, behav_vct):
    """
    Sorts rows/columns of a matrix according to a separate vector.
    """
    
    inds = behav_vct.argsort()
    mtx_sort = mtx
    mtx_sort = mtx_sort[inds, :]
    mtx_sort = mtx_sort[:, inds]
    
    return mtx_sort

def scale_mtx(mtx):
    """
    Scales a matrix to be between 0 and 1.
    """
    return (mtx-np.min(mtx))/(np.max(mtx)-np.min(mtx))

In [ ]:
## with AnnaK framework ###

n_subs = len(subj_data)
behav_sim_annak = np.zeros((n_subs, n_subs))

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([behav_rank[i], behav_rank[j]])/n_subs
            behav_sim_annak[i,j] = sim_ij
            behav_sim_annak[j,i] = sim_ij
        elif i==j:
            behav_sim_annak[i,j] = 1

behav_sim_annak = Adjacency(behav_sim_annak, matrix_type='similarity')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13,5))
behav_sim_annak.plot(axes=ax1)
ax1.set_title("Behavioral similarity matrix before sorting", fontsize=16)

sns.heatmap(sort_square_mtx(behav_sim_annak.squareform(), behav), ax = ax2, square=True)
ax2.set_title("Behavioral similarity matrix after sorting", fontsize=16)

In [ ]:
## IS-RSA r-value

isrsa_annak = similarity_matrix.similarity(behav_sim_annak, metric='spearman', n_permute=1, n_jobs=1)['correlation']
isrsa_annak

In [ ]:
# permutation test

stats = similarity_matrix.similarity(behav_sim_annak, metric='spearman', n_permute=5000, n_jobs=-1)
isrsa_r = stats['correlation']
isrsa_p = stats['p']

print(isrsa_r, isrsa_p)

### Network Lesion 

run with Shen 268 atlas

In [ ]:
## bring in FC data

data = []
for subj in subj_list:
    sub_data = []
    sub_data.append(pd.read_csv(os.path.join(data_dir, 'connectome',f'{subj}_FC_000.csv'), header=None, index_col=None, sep = ','))
    sub_data = pd.concat(sub_data)
    data.append(sub_data.values)
data = np.array(data)

print(data.shape)
n_subs, n_nodesx, n_nodesy = data.shape

In [ ]:
## bring in network label for each node

shen_label = pd.read_csv(os.path.join(data_dir, 'shen_268_10NETWORK.csv')) # can be any other atlas
shen_node = shen_label["Node"] # column name for node
shen_nodenet = shen_label["10_Network"] # column name for network label of each node

shen_node = np.array(shen_node)
print(shen_node.shape)
shen_nodenet = np.array(shen_nodenet)

In [ ]:
### leave within-network functional connectivities only ###

network_n = []

for sub in range(len(subj_list)):
    subconn = data[sub]
    subnet1 = []
    for kx in range(268):
        for ky in range(268):
            if shen_nodenet[kx] == shen_nodenet[ky] == 1 and kx > ky: # leave functional connectivites within a given(here, 1) network
                pass
            elif kx > ky:
                subnet1.append(subconn[kx, ky])
    network_n.append(subnet1)
            
for sub in range(len(network_n)):
    network_n[sub] = np.reshape(network_n[sub], (-1, 1)) # 2d array, matrix

network_n = np.array(network_n)

network_n.shape
    # output should be (number of sub, number of functional connectivities within a given network, 1)

In [ ]:
### connectome similarity matrix, metric = correlation ###

similarity_matrix = Adjacency(1-pairwise_distances(network_n[:, :, 0], metric='correlation'), matrix_type='similarity')
similarity_matrix.plot(cmap='RdBu_r')

and then run ISRSA above from here